In [3]:
import numpy as np
import networkx as nx  # tool to handle general Graphs 
import matplotlib.pyplot as plt 
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import itertools
import time
from datetime import datetime



from scipy.optimize import minimize

from numpy.random import choice
from numpy.random import normal
from qiskit import*

#classical_backend = Aer.get_backend('statevector_simulator')
from qiskit_algorithms.utils import algorithm_globals
from qiskit_ibm_runtime import Estimator, Sampler, Session, Options
from scipy.optimize import minimize
from qiskit_algorithms.optimizers import COBYLA
from qiskit_optimization.applications import Maxcut
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_algorithms import NumPyMinimumEigensolver

In [4]:
from qiskit_ibm_runtime import QiskitRuntimeService
 

QiskitRuntimeService.save_account(channel="ibm_quantum",
         token="....",
           set_as_default=True,overwrite=True)
 

service = QiskitRuntimeService()
backend = service.backend("ibm_sherbrooke")
backend = service.least_busy(operational=True, simulator=False)
backend.name


'ibm_osaka'

In [25]:




def min_cut_qiskit_QAOA(n_agents, induced_subgraph_game, reps , shots=2**12, seed_simulator=123, seed_transpiler=123, seed=123):
      global qp,heuristic,result,min_eigen_solver,sampler,session,options
      G=nx.Graph()
      G.add_nodes_from(np.arange(0,n_agents,1))
      elist = [tuple((int(x)-1 for x in key.split(',')))+tuple([induced_subgraph_game[key]*-1]) for key in induced_subgraph_game]
      G.add_weighted_edges_from(elist)
      w = [[G.get_edge_data(i,j,default = {'weight': 0})['weight']  for j in range(n_agents)] for i in range(n_agents)]
      w = np.array([np.array(row) for row in w])
      max_cut = Maxcut(w)
      qp = max_cut.to_quadratic_program()
      #operator, offset = qp.to_ising()
      algorithm_globals.random_seed = seed
      options = Options(optimization_level=3,resilience_level=1, environment={"log_level": "INFO","job_tags": [f'n_{n}idx_{idx}_estimator']})

      options.execution.shots = 2**12
      session = Session(backend=backend)
      sampler = Sampler(session=session, options=options)
      min_eigen_solver=QAOA(sampler=sampler, optimizer=COBYLA(maxiter=50000),callback=callback)
      heuristic =MinimumEigenOptimizer(min_eigen_solver)
      #heuristic =min_eigen_solver.compute_minimum_eigenvalue(operator)
      result = heuristic.solve(qp)
      #print('time',calculate_time_difference(session.details()['last_job_started'],session.details()['last_job_completed']))
      return result.x, -result.fval

In [19]:

rec = {'neval': [], 'param': [], 'res': [], 'metadata': []}
def callback(neval,param,res,metadata):
    """Record optimization history, in a global variable rec, which must be initialized"""
    global rec
    rec['neval'].append(neval)
    rec['param'].append(param)
    rec['res'].append(res)
    rec['metadata'].append(metadata)
    return

rec_2 = {'job_id': [], 'job_res': []}
def call_back_2(job_id,job_res):
  global rec_2
  print('job_id',job_id)
  print('job_res',job_res)
  rec_2['job_res'].append(job_id)
  rec_2['jon_id'].append(job_res)


def calculate_time_difference(start_time_str, end_time_str):
    # Define the format with fractional seconds
    format_with_fraction = "%Y-%m-%dT%H:%M:%S.%f"

    # Convert strings to datetime objects
    start_time = datetime.strptime(start_time_str[:-1], format_with_fraction)  # Removing the 'Z' at the end
    end_time = datetime.strptime(end_time_str[:-1], format_with_fraction)      # Removing the 'Z' at the end

    # Calculate the time difference in seconds
    time_difference = (end_time - start_time).total_seconds()

    return time_difference


In [ ]:

def evaluateSplits(coalition, coalition_values, **kwargs):
    #print("coalition",coalition,end='=')
    agents = coalition.split(',')
    n_agents = len(agents)
    best_cost_brute = f[coalition]
    xbest_brute = [coalition]
    for b in range(1, 2**(n_agents-1)):
        x = [int(term) for term in reversed(list(bin(b)[2:].zfill(n_agents)))]
        first_half = ','.join([agent for i,agent in enumerate(agents) if int(x[i])])
        second_half = ','.join([agent for i,agent in enumerate(agents) if not int(x[i])])
        if best_cost_brute <= (f[first_half]+f[second_half]):
            best_cost_brute = f[first_half]+f[second_half]
            xbest_brute = [first_half, second_half]
    #print(xbest_brute, best_cost_brute)
    return xbest_brute, best_cost_brute


def get_coalition_value(coalition, induced_subgraph_game):
    agents = coalition.split(',')
    return sum([induced_subgraph_game[','.join(map(str,sorted(map(int,key))))] for key in itertools.combinations(agents, 2)])



def evaluateSplits_min_cut(coalition, induced_subgraph_game, min_cut_solver = min_cut_qiskit_QAOA, **kwargs):
  #print("coalition",coalition,end='=')
    agents = coalition.split(',')
    n = len(agents)
    if n==1:
        return [coalition], 0
    if n==2:
        c_value = induced_subgraph_game[coalition]
        if c_value<=0:
            #print([agents[0],agents[1]], 0)
            return [agents[0],agents[1]], 0
        else:
            #print([coalition], c_value)
            return [coalition], c_value
    min_cut_mapping = {}
    for idx,agent in enumerate(agents):
        min_cut_mapping[agent] = str(idx+1)
    subproblem_as_induced_subgraph_game = {','.join([min_cut_mapping[vertex] for vertex in map(str,sorted(map(int,key)))]):induced_subgraph_game[','.join(map(str,sorted(map(int,key))))] for key in itertools.combinations(agents, 2)}
    xbest_brute, best_cost_brute = min_cut_solver(n,subproblem_as_induced_subgraph_game, **kwargs)
    if 0 in xbest_brute and 1 in xbest_brute:
        first_half = ','.join([agent for idx,agent in enumerate(agents) if xbest_brute[idx]])
        second_half = ','.join([agent for idx,agent in enumerate(agents) if not xbest_brute[idx]])
        bruteforce_solution_decoded = [first_half, second_half]
        best_cost_brute = get_coalition_value(first_half, induced_subgraph_game) + get_coalition_value(second_half, induced_subgraph_game)
    else:
        bruteforce_solution_decoded = [coalition]
        best_cost_brute = get_coalition_value(coalition, induced_subgraph_game)
    #print(bruteforce_solution_decoded, best_cost_brute)
    return bruteforce_solution_decoded, best_cost_brute


def IDP_min_cut_top_down(induced_subgraph_game, min_cut_solver = min_cut_qiskit_QAOA, **kwargs):
    grand_coalition = ','.join(map(str,sorted(map(int,(set([key.split(',')[i] for i in range(2) for key in induced_subgraph_game]))))))
    temp = [grand_coalition]
    optimal_cs = []
    while(len(temp)):
        c = temp.pop()
        c_split_t,c_split_f = evaluateSplits_min_cut(c, induced_subgraph_game, min_cut_solver = min_cut_qiskit_QAOA, **kwargs)
        if len(c_split_t)==1:
            optimal_cs+=c_split_t
        if len(c_split_t)>1:
            temp += c_split_t
    return optimal_cs, sum([get_coalition_value(c, induced_subgraph_game) for c in optimal_cs])


def generate_regular_graph(num_nodes, degree,Seed):
    np.random.seed(Seed)
    G = nx.random_regular_graph(degree, num_nodes,seed=Seed)
    #G=nx.erdos_renyi_graph(num_nodes, probability, seed=Seed)
    weights = np.random.normal(0,0.1,size=G.number_of_edges())

    edge_weights = {}
    for (u, v), weight in zip(G.edges(), weights):
        sorted_nodes = sorted([u+1, v+1])
        edge_key = f"{sorted_nodes[0]},{sorted_nodes[1]}"
        edge_weights[edge_key] = weight
    #print(edge_weights)
    # Assign zero weights for nodes that are not connected
    for u in range(1, num_nodes + 1):
        for v in range(u + 1, num_nodes + 1):
            sorted_nodes = sorted([u, v])
            edge_key = f"{sorted_nodes[0]},{sorted_nodes[1]}"
            if edge_key not in edge_weights:
                edge_weights[edge_key] = 0.0
    return edge_weights



In [ ]:
from datetime import datetime
import pickle
import itertools
import time
from Requirement import IDP_min_cut_top_down, evaluateSplits_min_cut,get_coalition_value,evaluateSplits
#algorithm_globals.massive=True
convert_ISG_to_coalition_game = True
# QAOA_p_increment_flag = True
#np.arange(4,14).tolist()


gate_based_solution_list=[]
gate_based_value_list=[]
gate_based_tte_list=[]

gate_based_quality_list=[]
result_abQAOA=[]
result_QAOA=[]

data= pickle.load(open('data_width_0.1.pkl', 'rb'))
Prosumers_list=list(data.keys())
# data=pickle.load(open('data_width_2.pkl', 'rb'))
# Prosumers_list=list(data.keys())
num_instances=20
MES_after2=([[[] for _ in range(num_instances)] for _ in range(len(Prosumers_list))])
MES_before2=([[[] for _ in range(num_instances)] for _ in range(len(Prosumers_list))])
MES_after1=([[[] for _ in range(num_instances)] for _ in range(len(Prosumers_list))])
MES_before1=([[[] for _ in range(num_instances)] for _ in range(len(Prosumers_list))])
depth=4
for n_idx,n in enumerate(Prosumers_list):
   
        for idx in range(num_instances):
            print(f'n. agents: {n}',end='\n')
            print('i',idx)
            
            induced_subgraph_game = data[n][idx]#generate_regular_graph(n, n-1,i)
            
            for PP in [1]:
                print(PP)
          
                start_time = time.time()
                #print(start_time)
                gate_based_solution, gate_based_value=( IDP_min_cut_top_down(induced_subgraph_game, \
                                                        min_cut_solver = min_cut_qiskit_QAOA,\
                                                                reps=PP))
                print(' gate_based_value', gate_based_value)
                
                gate_based_tte=((time.time() - start_time))
                gate_based_solution_list.append(gate_based_solution)
                gate_based_value_list.append(gate_based_value)
                gate_based_tte_list.append(gate_based_tte)

    
with open('gate_based_results.pkl', 'wb') as f:
    pickle.dump({
        'gate_based_solution_list': gate_based_solution_list,
        'gate_based_value_list': gate_based_value_list,
        'gate_based_tte_list': gate_based_tte_list
    }, f)